In [120]:
import requests
import pandas as pd
import os
import re
import math
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
from collections import Counter
from natsort import natsorted

In [163]:
episodes = []
title=[]
types = []
popularity = []
document=[]
# positioning
folder = r"./tsv_anime/"
# iter over the file
for anime in tqdm(natsorted(os.listdir(folder))):
    df = pd.read_csv(folder+anime, sep = "\t")
    episodes.append(df["animeNumEpisode"][0])
    types.append(df["animeType"][0])
    popularity.append(df["animePopularity"][0])
    document.append(df["animeDescription"][0])
    title.append(df["animeTitle"][0])
input_episodes=12
input_types="TV"
popularity=245
print(df)

100%|█████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:03<00:00, 480.48it/s]

               animeTitle animeType  animeNumEpisode releaseDate     endDate  \
0  Konjiki no Gash Bell!!        TV              150  2003-04-06  2006-03-26   

   animeNumMembers  animeScore  animeUsers  animeRank  animePopularity  \
0           107912        7.54       46265       1499             1519   

                                    animeDescription  \
0  Takamine Kiyomaro, a depressed don't-care-abou...   

                                        animeRelated  \
0  ['Konjiki no Gash!!', 'Konjiki no Gash Bell!!:...   

                                     animeCharacters  \
0  ['Bell, Gash', 'Takamine, Kiyomaro', 'Folgore,...   

                                         animeVoices  \
0  ['Ootani, IkueJapanese', 'Sakurai, TakahiroJap...   

                                          animeStaff  
0  [[['Hamano, Takatoshi', 'Producer'], ['Kaizawa...  


In [122]:
df

,animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff
0,Konjiki no Gash Bell!!,TV,150,2003-04-06,2006-03-26,107912,7.54,46265,1499,1519,"Takamine Kiyomaro, a depressed don't-care-abou...","['Konjiki no Gash!!', 'Konjiki no Gash Bell!!:...","['Bell, Gash', 'Takamine, Kiyomaro', 'Folgore,...","['Ootani, IkueJapanese', 'Sakurai, TakahiroJap...","[[['Hamano, Takatoshi', 'Producer'], ['Kaizawa..."


In [123]:
types

['TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'OVA',
 'TV',
 'Movie',
 'TV',
 'TV',
 'Movie',
 'TV',
 'TV',
 'Movie',
 'Movie',
 'TV',
 'TV',
 'TV',
 'Movie',
 'TV',
 'ONA',
 'TV',
 'TV',
 'TV',
 'Movie',
 'TV',
 'TV',
 'Movie',
 'TV',
 'TV',
 'TV',
 'ONA',
 'TV',
 'TV',
 'Movie',
 'TV',
 'TV',
 'TV',
 'TV',
 'OVA',
 'TV',
 'TV',
 'Movie',
 'TV',
 'Movie',
 'TV',
 'Movie',
 'TV',
 'Movie',
 'TV',
 'TV',
 'TV',
 'TV',
 'Movie',
 'TV',
 'TV',
 'TV',
 'TV',
 'Movie',
 'TV',
 'Movie',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'Movie',
 'Movie',
 'TV',
 'Movie',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'Movie',
 'TV',
 'Movie',
 'Special',
 'TV',
 'TV',
 'Movie',
 'Movie',
 'TV',
 'Movie',
 'TV',
 'TV',
 'Movie',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'TV',
 'Movie',
 'TV',
 'TV',
 'TV',
 'Movie',
 'TV',
 'ONA',
 'TV',
 'TV',
 'TV',
 'TV',
 'Movie',
 'TV',
 'TV',
 'Special',
 'Movie',
 'TV',
 'TV',
 'ONA',
 'TV',
 'Movie',
 'Movie',
 'Movie',
 'TV',
 'TV',
 'TV',
 '

In [64]:
def type_cosine(input_type,next_type):
    WORD = re.compile(r"\w+")
    intersection = set(input_type.keys()) & set(next_type.keys())
    numerator = sum([input_type[x] * next_type[x] for x in intersection])
    sum1 = sum([input_type[x] ** 2 for x in list(input_type.keys())])
    sum2 = sum([input_type[x] ** 2 for x in list(next_type.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator  

In [65]:
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [66]:
def type_score(input_type,next_type):
    vector1 = text_to_vector(input_type)
    vector2 = text_to_vector(next_type)
    cosine = type_cosine(vector1, vector2)
    return(cosine)

In [67]:
text1="Gintoki, Shinpachi, and Kagura return as the fun-loving but broke members of the Yorozuya team! Living in an alternate-reality Edo, where swords are prohibited and alien overlords have conquered Japan, they try to thrive on doing whatever work they can get their hands on. However, Shinpachi and Kagura still haven't been paid... Does Gin-chan really spend all that cash playing pachinko?\n\n\nMeanwhile, when Gintoki drunkenly staggers home one night, an alien spaceship crashes nearby. A fatally injured crew member emerges from the ship and gives Gintoki a strange, clock-shaped device, warning him that it is incredibly powerful and must be safeguarded. Mistaking it for his alarm clock, Gintoki proceeds to smash the device the next morning and suddenly discovers that the world outside his apartment has come to a standstill. With Kagura and Shinpachi at his side, he sets off to get the device fixed; though, as usual, nothing is ever that simple for the Yorozuya team.\n\n\nFilled with tongue-in-cheek humor and moments of heartfelt emotion, Gintama's fourth season finds Gintoki and his friends facing both their most hilarious misadventures and most dangerous crises yet.\n\n\n[Written by MAL Rewrite]"
text2='The self-proclaimed mad scientist Rintarou Okabe rents out a room in a rickety old building in Akihabara, where he indulges himself in his hobby of inventing prospective "future gadgets" with fellow lab members: Mayuri Shiina, his air-headed childhood friend, and Hashida Itaru, a perverted hacker nicknamed "Daru." The three pass the time by tinkering with their most promising contraption yet, a machine dubbed the "Phone Microwave," which performs the strange function of morphing bananas into piles of green gel.\n\n\nThough miraculous in itself, the phenomenon doesn\'t provide anything concrete in Okabe\'s search for a scientific breakthrough; that is, until the lab members are spurred into action by a string of mysterious happenings before stumbling upon an unexpected success—the Phone Microwave can send emails to the past, altering the flow of history.\n\n\nAdapted from the critically acclaimed visual novel by 5pb. and Nitroplus, Steins;Gate takes Okabe through the depths of scientific theory and practicality. Forced across the diverging threads of past and present, Okabe must shoulder the burdens that come with holding the key to the realm of time.\n\n\n[Written by MAL Rewrite]'
type_score(text1,text2)

0.7199269454413546

In [145]:
def identify_eps(input_eps,next_eps):
    if(type(next_eps) == str):
        return 0
    score=0
    #0 is film
    #1 is short
    #2 is medium
    #3 is large

    if(input_eps==1):
        category=0
    elif(input_eps<25 and input_eps>=8):
        category=1
    elif(input_eps<65 and input_eps>=25):
        category=2
    else:
        category=3
        
    if(next_eps==1):
        category1=0
    elif(next_eps<25 and input_eps>=8):
        category1=1
    elif(next_eps<65 and input_eps>=25):
        category1=2
    else:
        category1=3
    return(category,category1)

In [144]:
def episode_score(input_eps,next_eps):
    if(type(next_eps)==str):
        return 0
    category=(identify_eps(input_eps,next_eps))
    #print(category)
    if(category[0]==category[1]):
        return 1
    elif(abs(category[0]-category[1])==1):
        return 0.66
    elif(abs(category[0]-category[1])>1):
        return 0.33
    else:
        return 0

In [140]:
print(episode_score(27,63))

1


In [72]:
#def episode_score(input_eps,next_eps):
    #min_eps=1
    #max_eps=1000
    #diff=abs(input_eps-next_eps)
    #return diff

In [125]:
def final_score(i,input_type,next_type,input_eps,next_eps):
    print(i)
    score1=type_score(input_type,next_type)
    score2=episode_score(input_eps,next_eps)
    return(score1+score2)

In [128]:
len(episodes)

1500

In [131]:
for i in range(len(episodes)):
    print(episodes[i],type(episodes[i]))

64 <class 'numpy.int64'>
51 <class 'numpy.int64'>
10 <class 'numpy.int64'>
24 <class 'numpy.int64'>
13 <class 'numpy.int64'>
51 <class 'numpy.int64'>
148 <class 'numpy.int64'>
110 <class 'numpy.int64'>
13 <class 'numpy.int64'>
1 <class 'numpy.int64'>
12 <class 'numpy.int64'>
22 <class 'numpy.int64'>
1 <class 'numpy.int64'>
24 <class 'numpy.int64'>
201 <class 'numpy.int64'>
1 <class 'numpy.int64'>
1 <class 'numpy.int64'>
7 <class 'numpy.int64'>
16 <class 'numpy.int64'>
25 <class 'numpy.int64'>
1 <class 'numpy.int64'>
14 <class 'numpy.int64'>
11 <class 'numpy.int64'>
13 <class 'numpy.int64'>
12 <class 'numpy.int64'>
10 <class 'numpy.int64'>
1 <class 'numpy.int64'>
74 <class 'numpy.int64'>
13 <class 'numpy.int64'>
1 <class 'numpy.int64'>
26 <class 'numpy.int64'>
26 <class 'numpy.int64'>
26 <class 'numpy.int64'>
12 <class 'numpy.int64'>
12 <class 'numpy.int64'>
10 <class 'numpy.int64'>
1 <class 'numpy.int64'>
75 <class 'numpy.int64'>
24 <class 'numpy.int64'>
23 <class 'numpy.int64'>
24 <cl

In [143]:
type(episodes[70])

str

In [170]:
type_query="TV"
ep_query=13
scores=[]
for i in range(len(episodes)):
    score=final_score(i,type_query,types[i],ep_query,episodes[i])
    scores.append([score,i])
sorted_scores=sorted(scores, reverse=True)

for i in range(len(episodes)):
    #print(sorted_scores[i][1])
    print(sorted_scores[i],title[sorted_scores[i][1]])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

[0.66, 1322] Uchuu Kyoudai: Number Zero
[0.66, 1321] The iDOLM@STER Movie: Kagayaki no Mukougawa e!
[0.66, 1318] Sengoku Basara Movie: The Last Party
[0.66, 1316] Re:Zero kara Hajimeru Isekai Seikatsu - Hyouketsu no Kizuna
[0.66, 1315] Puparia
[0.66, 1314] Penguin Highway
[0.66, 1308] Lupin III: Episode 0 "First Contact"
[0.66, 1304] Kimi to, Nami ni Noretara
[0.66, 1302] Higashi no Eden: Air Communication
[0.66, 1301] Higashi no Eden Movie II: Paradise Lost
[0.66, 1296] Code Geass: Hangyaku no Lelouch Special Edition - Black Rebellion
[0.66, 1295] Bleach Movie 4: Jigoku-hen
[0.66, 1290] Ao no Exorcist Movie
[0.66, 1286] Kara no Kyoukai 1: Fukan Fuukei
[0.66, 1285] Juubee Ninpuuchou
[0.66, 1281] Evangelion: 3.0 You Can (Not) Redo
[0.66, 1280] Detective Conan Movie 09: Promo Special
[0.66, 1278] Boku no Hero Academia the Movie 2: Heroes:Rising - Epilogue Plus - Yume wo Genjitsu ni
[0.66, 1277] Yoru wa Honoka
[0.66, 1274] Towa no Quon 6: Towa no Quon
[0.66, 1273] Shoujo Kakumei Utena: Ad